In [1]:
import pandas as pd
from tqdm import tqdm
from sklearn import preprocessing

In [26]:
train_data = pd.read_csv('./train.csv')

In [27]:
train_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A


대출기간 : 36 months , 60 months   </br>
주택소유상태 : MORTGAGE,RENT,own,ANY </br>
대출목적 : 부채 통합 ,신용 카드,주택 개선 ,기타 ,주요 구매  ,의료 ,자동차,소규모 사업 ,이사 ,휴가, 주택 ,재생 에너지          
     
       

 

In [28]:
label_encoder = preprocessing.LabelEncoder()
train_data['대출기간'] = label_encoder.fit_transform(train_data['대출기간'] )
train_data['주택소유상태'] = label_encoder.fit_transform(train_data['주택소유상태'] )
train_data['대출목적'] = label_encoder.fit_transform(train_data['대출목적'] )


In [29]:
train_data.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,0,6 years,3,72000000,18.90,15,1,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,1,10+ years,1,130800000,22.33,21,10,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,0,5 years,1,96000000,8.60,14,1,0,928644,151944.0,0.0,0.0,A


근로기간

In [30]:
train_data['근로기간'].value_counts()

근로기간
10+ years    31585
2 years       8450
< 1 year      7774
3 years       7581
1 year        6249
Unknown       5671
5 years       5665
4 years       5588
8 years       4888
6 years       3874
7 years       3814
9 years       3744
10+years       896
<1 year        370
3               89
1 years         56
Name: count, dtype: int64

In [31]:
train_data['근로기간'] = train_data['근로기간'].str.extract(r'(\d+)')
train_data['근로기간']  =train_data['근로기간'].fillna(0)
train_data['근로기간']  =train_data['근로기간'].astype(int)

main processing

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [33]:
train_data = train_data.drop(columns='ID')
train_data_target = train_data['대출등급']
train_data = train_data.drop(columns='대출등급')

In [34]:
train_data_x, valid_data_x, train_data_y, valid_data_y= train_test_split(train_data,train_data_target,test_size=0.2, shuffle=True,random_state=34)

In [23]:
# # model = DecisionTreeClassifier(random_state=42)
# model = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=42)
# model.fit(train_data_x, train_data_y)

# # validation set을 사용하여 모델 예측
# y_pred = model.predict(valid_data_x)

# # 모델 평가 (정확도)
# accuracy = accuracy_score(valid_data_y, y_pred)
# print(f'모델 정확도: {accuracy:.2f}')

모델 정확도: 0.83


In [35]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[500, 700, 1000],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18],
    'min_samples_split' : [8, 16, 20]
}

rf_clf = RandomForestClassifier(criterion='entropy', random_state=42, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=2, n_jobs=2, verbose=2 )
grid_cv.fit(train_data_x , train_data_y) # grid.cv.fit(train_x, train_y)

estimator =grid_cv.best_estimator_ 
pred = estimator.predict(valid_data_x) # estimator.predict(test)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))
accuracy = accuracy_score(valid_data_y, pred)
print(f'모델 정확도: {accuracy:.2f}')

Fitting 2 folds for each of 108 candidates, totalling 216 fits


KeyboardInterrupt: 

valid set 안쓰기

In [18]:
model = DecisionTreeClassifier(random_state=42)
model.fit(train_data, train_data_target)

DecisionTreeClassifier(random_state=42)

test

In [19]:
test_data = pd.read_csv('./test.csv')
test_data.head(2)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수
0,TEST_00000,16800000,36 months,8 years,MORTGAGE,132000000,19.64,12,주택 개선,0,394692,146604.0,0.0,0.0
1,TEST_00001,8400000,36 months,5 years,RENT,89971200,15.84,25,부채 통합,0,0,0.0,0.0,0.0


In [20]:
test_data['대출기간'] = label_encoder.fit_transform(test_data['대출기간'] )
test_data['주택소유상태'] = label_encoder.fit_transform(test_data['주택소유상태'] )
test_data['대출목적'] = label_encoder.fit_transform(test_data['대출목적'] )
test_data['근로기간'] = test_data['근로기간'].str.extract(r'(\d+)')
test_data['근로기간']  =test_data['근로기간'].fillna(0)
test_data['근로기간']  =test_data['근로기간'].astype(int)

In [21]:
test_data = test_data.drop(columns='ID')

In [22]:
test_pred = model.predict(test_data)

submissionfile 생성

In [23]:
submission_data = pd.read_csv('./sample_submission.csv')
submission_data['대출등급'] = test_pred
submission_data.to_csv('./decision_tree.csv', index=False)


In [24]:
submission_data

,ID,대출등급
0,TEST_00000,B
1,TEST_00001,C
2,TEST_00002,A
3,TEST_00003,C
4,TEST_00004,C
...,...,...
64192,TEST_64192,D
64193,TEST_64193,D
64194,TEST_64194,D
64195,TEST_64195,C
